In [1]:
# Install packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 11.4 MB/s eta 0:00:00


In [2]:
!pip install -qU langchain-classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY_DEFAULT")
print(" API key set.")


 API key set.


In [4]:
from typing import Dict
from datetime import datetime
from zoneinfo import ZoneInfo

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI       # to work with Gemini models
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor, create_react_agent


In [5]:
# Small in-memory dictionary (data source) can be replaced with a real API
MOCK_WEATHER: Dict[str, Dict[str, str]] = {
    "mumbai":  {"condition": "Humid, partly cloudy", "temp_c": "31", "humidity": "75%", "rain_chance": "40%"},
    "delhi":   {"condition": "Hot, dry",             "temp_c": "34", "humidity": "35%", "rain_chance": "10%"},
    "hyderabad":{"condition": "Warm, breezy",        "temp_c": "30", "humidity": "55%", "rain_chance": "20%"},
    "bengaluru":{"condition": "Mild, cloudy",        "temp_c": "26", "humidity": "65%", "rain_chance": "30%"},
}


In [6]:
# Return the current time stamp in ist, responses look local
def _now_ist_str() -> str:
    ist = ZoneInfo("Asia/Kolkata")
    return datetime.now(ist).strftime("%Y-%m-%d %H:%M IST")


Accept city name as input, return a short weather report, if no cit is provided, ask the user to provide one
Check the city name against the small dataset (dict)

In [7]:
@tool
def get_weather(city: str) -> str:
    """Return a short, human-friendly weather report for the given city (mock data)."""
    if not city or not city.strip():
        return "Please provide a city name."
    key = city.strip().lower()
    data = MOCK_WEATHER.get(key)
    if not data:
        # Sensible default when city isn't in our mock set
        data = {"condition": "Partly cloudy", "temp_c": "30", "humidity": "60%", "rain_chance": "20%"}
    return (
        f"City: {city.strip().title()}\n"
        f"Time: {_now_ist_str()}\n"
        f"Condition: {data['condition']}\n"
        f"Temperature: {data['temp_c']} °C\n"
        f"Humidity: {data['humidity']}\n"
        f"Chance of rain: {data['rain_chance']}"
    )


In [12]:
def build_weather_agent():
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    tools = [get_weather]

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a helpful weather assistant.\n"
         "You have access to the following tools:\n"
         "{tools}\n\n"  # Placeholder for tool descriptions
         "Use the following format:\n\n"
         "Question: the input question you must answer\n"
         "Thought: you should always think about what to do\n"
         "Action: the action to take, should be one of [{tool_names}]\n" # Placeholder for tool names
         "Action Input: the input to the action\n"
         "Observation: the result of the action\n"
         "... (this Thought/Action/Action Input/Observation can repeat N times)\n"
         "Thought: I now know the final answer\n"
         "Final Answer: the final answer to the original input question\n\n"
         "Begin!\n\n"
         "RULES:\n"
         "1) Always call the get_weather tool to answer weather questions.\n"
         "2) If the city is missing or unclear, ask the user to specify a city.\n"
         "3) Return concise, friendly answers.\n"),
        ("human", "{input}\n{agent_scratchpad}"), # agent_scratchpad as a string here
    ])

    agent = create_react_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)

weather_agent = build_weather_agent()
print(" Weather agent ready.")

 Weather agent ready.


In [15]:
def ask(q: str):
    res = weather_agent.invoke({"input": q})
    print(res["output"], "\n")


In [16]:
# Known city
ask("What’s the weather in Mumbai?")

The weather in Mumbai is humid and partly cloudy with a temperature of 31°C. There's a 40% chance of rain and humidity is at 75%. 



In [17]:
# Another known city
ask("Weather update for Bengaluru, please.")

The weather in Bengaluru is currently mild and cloudy with a temperature of 26°C. There's a 30% chance of rain, and humidity is at 65%. 

